# The Movie DataBase (TMDB)
## Foundations Of Computer Science Exam Project Stefano Daraio

Starting from the The Movie DataBase (TMDB) sample datasets,

1. For each movie, compute the number of cast members
2. How many movies do not have a homepage?
3. For each year, how many movies do not have a homepage?
4. Extract the domain of each homepage.
5. Extract a set of normalized tables. That is, each entry of a normalized table must contain exactly one value (not a list or a dictionary).
6. For each movie, compute the gross margin (difference between revenue and budget)
7. For each movie, compute the number of crew members
8. For each movie, compute the number of directors
9. For each language, compute the number of movies where such language is spoken.
10. For each company and each decade, compute the overall revenue
11. For each decade, compute the company with maximum revenue
12. In each year, how many movies have revenue smaller than the budget?

In [2]:
import pandas as pd
import re
import numpy as np
import json


In [120]:
movies = pd.read_csv('tmdb_5000_movies.csv')
# movies.head(2)
movies.isnull 

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
3730,0,"[{""id"": 53, ""name"": ""Thriller""}, {""id"": 9648, ...",http://www.cargoderfilm.ch http://cargothemovi...,34069,"[{""id"": 3388, ""name"": ""space colony""}, {""id"": ...",de,Cargo,The story of CARGO takes place on rusty space-...,10.142218,"[{""name"": ""Telepool"", ""id"": 823}, {""name"": ""At...","[{""iso_3166_1"": ""CH"", ""name"": ""Switzerland""}]",2009-09-24,0,120.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,NaN,Cargo,5.9,140
3731,4200000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://tv.disney.go.com/disneychannel/original...,10947,"[{""id"": 1412, ""name"": ""becoming an adult""}, {""...",en,High School Musical,"Troy (Zac Efron), the popular captain of the b...",16.536374,"[{""name"": ""Disney Channel"", ""id"": 3213}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2006-01-20,0,98.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,This School Rocks Like No Other!,High School Musical,6.1,1000
3732,0,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 10749, ""n...",NaN,47452,"[{""id"": 1523, ""name"": ""obsession""}, {""id"": 967...",en,Love and Death on Long Island,Giles De'Ath is a widower who doesn't like any...,0.933891,"[{""name"": ""Arts Council of England"", ""id"": 718...","[{""iso_3166_1"": ""CA"", ""name"": ""Canada""}, {""iso...",1997-01-01,0,93.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,Love and Death on Long Island,6.9,11
3733,4200000,"[{""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ""na...",NaN,3040,"[{""id"": 616, ""name"": ""witch""}, {""id"": 1552, ""n...",ru,Ночной дозор,"Among normal humans live the ""Others"" possessi...",12.540624,"[{""name"": ""Bazelevs Production"", ""id"": 1038}, ...","[{""iso_3166_1"": ""RU"", ""name"": ""Russia""}]",2004-06-27,32000000,114.0,"[{""iso_639_1"": ""de"", ""name"": ""Deutsch""}, {""iso...",Released,All That Stands Between Light And Darkness Is ...,Night Watch,6.3,210
3734,3705538,"[{""id"": 10749, ""name"": ""Romance""}, {""id"": 80, ...",http://www.miramax.com/movie/the-crying-game/,11386,"[{""id"": 108, ""name"": ""transvestism""}, {""id"": 2...",en,The Crying Game,Irish Republican Army member Fergus (Stephen R...,8.839651,"[{""name"": ""Miramax Films"", ""id"": 14}, {""name"":...","[{""iso_3166_1"": ""JP"", ""name"": ""Japan""}, {""iso_...",1992-09-02,62548947,112.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Play At Your Own Risk.,The Crying Game,6.9,176
3735,4000000,"[{""id"": 35, ""name"": ""Comedy""}]",NaN,10246,"[{""id"": 255, ""name"": ""male nudity""}, {""id"": 29...",en,Porky's,"Set in 1954, a group of Florida high schoolers...",12.891276,"[{""name"": ""Melvin Simon Productions"", ""id"": 21...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",1981-11-13,125728258,94.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Keep an eye out for the funniest movie about g...,Porky's,6.1,183
3736,4000000,"[{""id"": 27, ""name"": ""Horror""}, {""id"": 878, ""na...",http://magnetreleasing.com/survivalofthedead/,29426,"[{""id"": 12377, ""name"": ""zombie""}]",en,Survival of the Dead,"On an island off the coast of North America, l...",10.789912,"[{""name"": ""Devonshire Productions"", ""id"": 2391...","[{""iso_3166_1"": ""CA"", ""name"": ""Canada""}, {""iso...",2010-05-06,0,90.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Death isn't what it used to be.,Survival of the Dead,4.6,116
3737,114000,"[{""id"": 27, ""name"": ""Horror""}]",NaN,10331,"[{""id"": 1155, ""name"": ""brother sister relation...",en,Night of the Living Dead,A group of people try to survive an attack of ...,25.666029,"[{""name"": ""Laurel Group"", ""id"": 638}, {""name"":...","[{""iso_3166_1"": ""US"", ""name"": ""United States o.

In [98]:
credits = pd.read_csv('tmdb_5000_credits.csv')
credits.head(10)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."
5,559,Spider-Man 3,"[{""cast_id"": 30, ""character"": ""Peter Parker / ...","[{""credit_id"": ""52fe4252c3a36847f80151a5"", ""de..."
6,38757,Tangled,"[{""cast_id"": 34, ""character"": ""Flynn Rider (vo...","[{""credit_id"": ""52fe46db9251416c91062101"", ""de..."
7,99861,Avengers: Age of Ultron,"[{""cast_id"": 76, ""character"": ""Tony Stark / Ir...","[{""credit_id"": ""55d5f7d4c3a3683e7e0016eb"", ""de..."
8,767,Harry Potter and the Half-Blood Prince,"[{""cast_id"": 3, ""character"": ""Harry Potter"", ""...","[{""credit_id"": ""52fe4273c3a36847f801fab1"", ""de..."
9,209112,Batman v Superman: Dawn of Justice,"[{""cast_id"": 18, ""character"": ""Bruce Wayne / B...","[{""credit_id"": ""553bf23692514135c8002886"", ""de..."


### 1) For each movie, compute the number of cast members

In [99]:
def cast_n(data):
    item_dict = json.loads(data)
    return(len(item_dict))
credits['cast_n'] = credits['cast'].apply(cast_members)
credits[['movie_id','title','cast_n']].head(10)

,movie_id,title,cast_n
0,19995,Avatar,83
1,285,Pirates of the Caribbean: At World's End,34
2,206647,Spectre,83
3,49026,The Dark Knight Rises,158
4,49529,John Carter,27
5,559,Spider-Man 3,143
6,38757,Tangled,13
7,99861,Avengers: Age of Ultron,72
8,767,Harry Potter and the Half-Blood Prince,49
9,209112,Batman v Superman: Dawn of Justice,152


### 2. How many movies do not have a homepage?

In [17]:
conteggio = len(movies[movies['homepage'].isnull()])
print('i film che non hanno homepage sono',conteggio)

i film che non hanno homepage sono 3091


### 3. For each year, how many movies do not have a homepage?

In [145]:
# creo variabile anno
movies['years'] = pd.to_datetime(movies['release_date'],format="%Y-%m-%d").dt.year
movies['years'] = movies['years'].replace(np.nan, 'Missing')
#selezione film senza homepage
nohomepage = movies[movies['homepage'].isnull()]
nohomepage[['id', 'years']].groupby('years').count().tail().rename(index=str, columns={'id':'count_noHomepage'})

,count_noHomepage
years,
2013.0,127
2014.0,157
2015.0,110
2016.0,31
Missing,1


### 4. Extract the domain of each homepage.

In [143]:
movies.dom = [re.findall('://([\w+\.+\w+]*)', i) for i in movies['homepage'] if pd.notnull(i)] #list comprehension with regex
movies[['title', 'homepage', 'domain']].head(10)

,title,homepage,domain
0,Avatar,http://www.avatarmovie.com/,www.avatarmovie.com
1,Pirates of the Caribbean: At World's End,http://disney.go.com/disneypictures/pirates/,disney.go.com
2,Spectre,http://www.sonypictures.com/movies/spectre/,www.sonypictures.com
3,The Dark Knight Rises,http://www.thedarkknightrises.com/,www.thedarkknightrises.com
4,John Carter,http://movies.disney.com/john-carter,movies.disney.com
5,Spider-Man 3,http://www.sonypictures.com/movies/spider-man3/,www.sonypictures.com
6,Tangled,http://disney.go.com/disneypictures/tangled/,disney.go.com
7,Avengers: Age of Ultron,http://marvel.com/movies/movie/193/avengers_ag...,marvel.com
8,Harry Potter and the Half-Blood Prince,http://harrypotter.warnerbros.com/harrypottera...,harrypotter.warnerbros.com
9,Batman v Superman: Dawn of Justice,http://www.batmanvsupermandawnofjustice.com/,www.batmanvsupermandawnofjustice.com


### 5. Extract a set of normalized tables. That is, each entry of a normalized table must contain exactly one value (not a list or a dictionary).

#### Create movie_id

In [147]:
movie_id = movies[['id', 'budget', 'homepage', 'domain', 'original_language', 'original_title', 'overview', 'popularity', 'release_date','years', 'revenue', 'runtime', 'status', 'tagline', 'title', 'vote_average', 'vote_count' ]]
movie_id = movie_id.rename(index=str, columns={'id': 'movie_id'})
movie_id = movie_id.set_index(['movie_id'])
movie_id.head(5)

,budget,homepage,domain,original_language,original_title,overview,popularity,release_date,years,revenue,runtime,status,tagline,title,vote_average,vote_count
movie_id,,,,,,,,,,,,,,,,
19995,237000000,http://www.avatarmovie.com/,www.avatarmovie.com,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,2009-12-10,2009,2787965087,162.0,Released,Enter the World of Pandora.,Avatar,7.2,11800
285,300000000,http://disney.go.com/disneypictures/pirates/,disney.go.com,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,2007-05-19,2007,961000000,169.0,Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
206647,245000000,http://www.sonypictures.com/movies/spectre/,www.sonypictures.com,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,2015-10-26,2015,880674609,148.0,Released,A Plan No One Escapes,Spectre,6.3,4466
49026,250000000,http://www.thedarkknightrises.com/,www.thedarkknightrises.com,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,2012-07-16,2012,1084939099,165.0,Released,The Legend Ends,The Dark Knight Rises,7.6,9106
49529,260000000,http://movies.disney.com/john-carter,movies.disney.com,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,2012-03-07,2012,284139100,132.0,Released,"Lost in our world, found in another.",John Carter,6.1,2124


#### Create genre_id

In [210]:
genere_id = pd.DataFrame()
for film in list(range(0,len(movies['genres']))):
    tabjson = pd.read_json(movies['genres'][film])
    idfilm = movies['id'][film]
    tabjson['movie_id'] = idfilm
    genere_id = pd.concat([genere_id, tabjson])
genere_id = genere_id.rename(index=str, columns={'id': 'genre_id'})
genere_id = genere_id.rename(index=str, columns={'name': 'genre_name'})
genere_id = genere_id[['movie_id','genre_id', 'genre_name']].set_index(['movie_id','genre_id', 'genre_name'])
genere_id.head(5)

Empty DataFrame
Columns: []
Index: [(19995, 28.0, Action), (19995, 12.0, Adventure), (19995, 14.0, Fantasy), (19995, 878.0, Science Fiction), (285, 12.0, Adventure)]

#### Create keywords_id

In [216]:
keywords_id = pd.DataFrame()
for film in list(range(0,len(movies['keywords']))):
    tabjson = pd.read_json(movies['keywords'][film])
    idfilm = movies['id'][film]
    tabjson['movie_id'] = idfilm
    keywords_id = pd.concat([keywords_id, tabjson])
keywords_id = keywords_id.rename(index=str, columns={'id': 'key_id'})
keywords_id = keywords_id.rename(index=str, columns={'name': 'key_name'})
keywords = keywords_id[['key_id', 'key_name']]
keywords_id = keywords_id[['movie_id', 'key_id', 'key_name']].set_index(['movie_id', 'key_id', 'key_name'])
keywords_id.head(5)

Empty DataFrame
Columns: []
Index: [(19995, 1463.0, culture clash), (19995, 2964.0, future), (19995, 3386.0, space war), (19995, 3388.0, space colony), (19995, 3679.0, society)]

In [217]:
prod_companies_id = pd.DataFrame()
for film in list(range(0,len(movies['production_companies']))):
    tabjson = pd.read_json(movies['production_companies'][film])
    idfilm = movies['id'][film]
    tabjson['movie_id'] = idfilm
    prod_companies_id = pd.concat([prod_companies_id, tabjson])
prod_companies_id = prod_companies_id.rename(index=str, columns={'id': 'company_id'})
prod_companies_id = prod_companies_id.rename(index=str, columns={'name': 'company_name'})
prod_companies = prod_companies_id[['company_id', 'company_name']]
prod_companies_id = prod_companies_id[['movie_id', 'company_id', 'company_name']].set_index(['movie_id', 'company_id', 'company_name'])
prod_companies_id.head(5)

Empty DataFrame
Columns: []
Index: [(19995, 289.0, Ingenious Film Partners), (19995, 306.0, Twentieth Century Fox Film Corporation), (19995, 444.0, Dune Entertainment), (19995, 574.0, Lightstorm Entertainment), (285, 2.0, Walt Disney Pictures)]

In [218]:
prod_countries_id = pd.DataFrame()
for film in list(range(0,len(movies['production_countries']))):
    tabjson = pd.read_json(movies['production_countries'][film])
    idfilm = movies['id'][film]
    tabjson['movie_id'] = idfilm
    prod_countries_id = pd.concat([prod_countries_id, tabjson])
prod_countries_id = prod_countries_id.rename(index=str, columns={'iso_3166_1': 'country_id'})
prod_countries_id = prod_countries_id.rename(index=str, columns={'name': 'country_name'})
prod_countries = prod_countries_id[['country_id', 'country_name']]
prod_countries_id = prod_countries_id[['movie_id', 'country_id', 'country_name']].set_index(['movie_id', 'country_id', 'country_name'])
prod_countries_id.head()

Empty DataFrame
Columns: []
Index: [(19995, US, United States of America), (19995, GB, United Kingdom), (285, US, United States of America), (206647, GB, United Kingdom), (206647, US, United States of America)]

In [219]:
spoken_languages_id = pd.DataFrame()
for film in list(range(0,len(movies['spoken_languages']))):
    tabjson = pd.read_json(movies['spoken_languages'][film])
    idfilm = movies['id'][film]
    tabjson['movie_id'] = idfilm
    spoken_languages_id = pd.concat([spoken_languages_id, tabjson])
spoken_languages_id = spoken_languages_id.rename(index=str, columns={'iso_639_1': 'language_id'})
spoken_languages_id = spoken_languages_id.rename(index=str, columns={'name': 'language_name'})
spoken_languages = spoken_languages_id[['language_id', 'language_name']]
spoken_languages_id = spoken_languages_id[['movie_id', 'language_id', 'language_name']].set_index(['movie_id', 'language_id', 'language_name'])
spoken_languages_id.head()

Empty DataFrame
Columns: []
Index: [(19995, en, English), (19995, es, Español), (285, en, English), (206647, fr, Français), (206647, en, English)]

In [222]:
cast = pd.DataFrame()
for film in list(range(0,len(credits['cast']))):
    tabjson = pd.read_json(credits['cast'][film])
    idfilm = credits['movie_id'][film]
    tabjson['movie_id'] = idfilm
    cast = pd.concat([cast, tabjson])
cast = cast.rename(index=str, columns={'id': 'pers_id'})
cast_id = cast[['movie_id', 'cast_id', 'pers_id', 'order', 'credit_id', 'character']]
cast_id = cast_id.set_index(['credit_id'])
cast_id.head()

,movie_id,cast_id,pers_id,order,character
credit_id,,,,,
5602a8a7c3a3685532001c9a,19995,242.0,65731.0,0.0,Jake Sully
52fe48009251416c750ac9cb,19995,3.0,8691.0,1.0,Neytiri
52fe48009251416c750aca39,19995,25.0,10205.0,2.0,Dr. Grace Augustine
52fe48009251416c750ac9cf,19995,4.0,32747.0,3.0,Col. Quaritch
52fe48009251416c750ac9d3,19995,5.0,17647.0,4.0,Trudy Chacon


In [223]:
crew = pd.DataFrame()
for film in list(range(0,len(credits['crew']))):
    tabjson = pd.read_json(credits['crew'][film])
    idfilm = credits['movie_id'][film]
    tabjson['movie_id'] = idfilm
    crew = pd.concat([crew, tabjson])
crew = crew.rename(index=str, columns={'id': 'pers_id'})
crew_id = crew[['movie_id', 'credit_id', 'pers_id', 'department', 'job']]
crew_id = crew_id.set_index(['credit_id'])
crew_id.head()

,movie_id,pers_id,department,job
credit_id,,,,
52fe48009251416c750aca23,19995,1721.0,Editing,Editor
539c47ecc3a36810e3001f87,19995,496.0,Art,Production Design
54491c89c3a3680fb4001cf7,19995,900.0,Sound,Sound Designer
54491cb70e0a267480001bd0,19995,900.0,Sound,Supervising Sound Editor
539c4a4cc3a36810c9002101,19995,1262.0,Production,Casting


In [254]:
attori_crew = crew[['pers_id', 'gender', 'name']]
attori_cast = cast[['pers_id', 'gender', 'name']]
attori = pd.concat([attori_crew, attori_cast])
attori = attori.sort_values('pers_id').drop_duplicates().set_index('pers_id')
attori.head(5)

,gender,name
pers_id,,
1.0,2.0,George Lucas
2.0,2.0,Mark Hamill
3.0,2.0,Harrison Ford
4.0,1.0,Carrie Fisher
5.0,2.0,Peter Cushing


### 6. For each movie, compute the gross margin (difference between revenue and budget)

In [257]:
movie_id['gross_margin'] = movie_id['revenue'] - movie_id['budget']

In [260]:
movie_id
movie_id.sort_values(['gross_margin'], ascending=[False])

,budget,homepage,domain,original_language,original_title,overview,popularity,release_date,years,revenue,runtime,status,tagline,title,vote_average,vote_count,gross_margin
movie_id,,,,,,,,,,,,,,,,,
19995,237000000,http://www.avatarmovie.com/,www.avatarmovie.com,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,2009-12-10,2009,2787965087,162.0,Released,Enter the World of Pandora.,Avatar,7.2,11800,2550965087
597,200000000,http://www.titanicmovie.com,www.titanicmovie.com,en,Titanic,"84 years later, a 101-year-old woman named Ros...",100.025899,1997-11-18,1997,1845034188,194.0,Released,Nothing on Earth could come between them.,Titanic,7.5,7562,1645034188
135397,150000000,http://www.jurassicworld.com/,www.jurassicworld.com,en,Jurassic World,Twenty-two years after the events of Jurassic ...,418.708552,2015-06-09,2015,1513528810,124.0,Released,The park is open.,Jurassic World,6.5,8662,1363528810
168259,190000000,http://www.furious7.com/,www.furious7.com,en,Furious 7,Deckard Shaw seeks revenge against Dominic Tor...,102.322217,2015-04-01,2015,1506249360,137.0,Released,Vengeance Hits Home,Furious 7,7.3,4176,1316249360
24428,220000000,http://marvel.com/avengers_movie/,marvel.com,en,The Avengers,When an unexpected enemy emerges and threatens...,144.448633,2012-04-25,2012,1519557910,143.0,Released,Some assembly required.,The Avengers,7.4,11776,1299557910
99861,280000000,http://marvel.com/movies/movie/193/avengers_ag...,marvel.com,en,Avengers: Age of Ultron,When Tony Stark tries to jumpstart a dormant p...,134.279229,2015-04-22,2015,1405403694,141.0,Released,A New Age Has Come.,Avengers: Age of Ultron,7.3,6767,1125403694
109445,150000000,http://movies.disney.com/frozen,movies.disney.com,en,Frozen,Young princess Anna of Arendelle dreams about ...,165.125366,2013-11-27,2013,1274219009,102.0,Released,Only the act of true love will thaw a frozen h...,Frozen,7.3,5295,1124219009
211672,74000000,http://www.minionsmovie.com/,www.minionsmovie.com,en,Minions,"Minions Stuart, Kevin and Bob are recruited by...",875.581305,2015-06-17,2015,1156730962,91.0,Released,"Before Gru, they had a history of bad bosses",Minions,6.4,4571,1082730962
122,94000000,http://www.lordoftherings.net,www.lordoftherings.net,en,The Lord of the Rings: The Return of the King,Aragorn is revealed as the heir to the ancient...,123.630332,2003-12-01,2003,1118888979,201.0,Released,The eye of the enemy is moving.,The Lord of the Rings: The Return of the King,8.1,8064,1024888979


### 7. For each movie, compute the number of crew members

In [275]:
n_crew = crew[['movie_id','pers_id']].groupby('movie_id').count()
n_crew = members_movie.rename(index=str, columns={'pers_id': 'n_crew'})
n_crew.head()


,n_crew
movie_id,
5,88
11,20
12,104
13,93
14,109


### 8. For each movie, compute the number of directors

In [320]:
directors = crew.loc[crew['job'] == 'Director']
num_directors = directors[['movie_id', 'pers_id']].groupby('movie_id').count()
num_directors = pd.merge(num_directors.reset_index(), movie_id.reset_index(), on='movie_id') 
num_directors = num_directors.rename(index=str, columns={'pers_id': 'n_directors'})
num_directors[['movie_id', 'title', 'n_directors']].set_index('movie_id').head()

,title,n_directors
movie_id,,
5,Four Rooms,4
11,Star Wars,1
12,Finding Nemo,1
13,Forrest Gump,1
14,American Beauty,1


### 9. For each language, compute the number of movies where such language is spoken.

In [349]:
lang = spoken_languages_id.reset_index().groupby(['language_name','language_id']).count()
lang = lang.iloc[27:]
lang = lang.rename(index=str, columns={'movie_id': 'n_movie'})
#lang.head(50)
lang.sort_values(['n_movie'], ascending=[False])

,,n_movie
language_name,language_id,
English,en,4485
Français,fr,437
Español,es,351
Deutsch,de,262
Italiano,it,188
Pусский,ru,185
普通话,zh,107
日本語,ja,97
Português,pt,68


### 10. For each company and each decade, compute the overall revenue

In [393]:
#si calcolano il numero di compagnie per film
pc_id = prod_companies_id.reset_index()
pc_counter = pc_id.groupby('movie_id').count()
pc_counter.head(5)

,company_id,company_name
movie_id,,
5,2,2
11,2,2
12,1,1
13,1,1
14,2,2


In [394]:


#calcolo della decade di ogni film
movie_id['decade'] = pd.to_datetime(movie_id['release_date'], format="%Y-%m-%d").dt.year//10*10
movie_id['decade'] = movie_id.decade.replace(np.nan, 'Missing')
movie_id.sort_values(['release_date'], ascending=[True]).head(3)

,budget,homepage,domain,original_language,original_title,overview,popularity,release_date,years,revenue,runtime,status,tagline,title,vote_average,vote_count,gross_margin,decade
movie_id,,,,,,,,,,,,,,,,,,
3059,385907,NaN,NaN,en,Intolerance,"The story of a poor young woman, separated by ...",3.232447,1916-09-04,1916,8394751,197.0,Released,The Cruel Hand of Intolerance,Intolerance,7.4,60,8008844,1910
3060,245000,NaN,NaN,en,The Big Parade,The story of an idle rich boy who joins the US...,0.785744,1925-11-05,1925,22000000,151.0,Released,NaN,The Big Parade,7.0,21,21755000,1920
19,92620000,NaN,NaN,de,Metropolis,In a futuristic city sharply divided between t...,32.351527,1927-01-10,1927,650422,153.0,Released,There can be no understanding between the hand...,Metropolis,8.0,657,-91969578,1920


In [397]:
#calcolo le revenue per ogni film e le revenue per ogni film/company
pc = pd.merge(pc_counter.reset_index(), movie_id.reset_index(), on='movie_id')
pc = pc[['movie_id', 'company_id', 'revenue', 'decade']].rename(index=str, columns={'company_id':'count_companies'})
pc['revenue/companies'] = pc['revenue'] / pc['count_companies']
pc.head(3)

,movie_id,count_companies,revenue,decade,revenue/companies
0,5,2,4300000,1990,2150000.0
1,11,2,775398007,1970,387699003.5
2,12,1,940335536,2000,940335536.0


In [402]:
#guadagno per ogni film/compagnia
revenue = pd.merge(pc_id, pc[['movie_id', 'decade', 'revenue', 'count_companies', 'revenue/companies']], on='movie_id')
revenue.head()

,movie_id,company_id,company_name,decade,revenue,count_companies,revenue/companies
0,19995,289.0,Ingenious Film Partners,2000,2787965087,4,6.969913e+08
1,19995,306.0,Twentieth Century Fox Film Corporation,2000,2787965087,4,6.969913e+08
2,19995,444.0,Dune Entertainment,2000,2787965087,4,6.969913e+08
3,19995,574.0,Lightstorm Entertainment,2000,2787965087,4,6.969913e+08
4,285,2.0,Walt Disney Pictures,2000,961000000,3,3.203333e+08


In [444]:
#guadagno totale per ogni compagnia/decade
revenue_grouped = revenue[['company_id','company_name', 'decade', 'revenue/companies']].groupby(['company_id', 'company_name','decade']).sum().rename(index=str, columns={'revenue/companies':'overall_revenue'})
revenue_grouped.head(9)

overall_revenue
company_id company_name         decade                 
1.0        Lucasfilm            1970.0     4.343657e+08
                                1980.0     1.187157e+09
                                1990.0     9.243176e+08
                                2000.0     1.892716e+09
                                2010.0     1.678846e+07
2.0        Walt Disney Pictures 1940.0     8.332000e+07
                                1990.0     1.909229e+09
                                2000.0     4.803996e+09
                                2010.0     6.003551e+09

### 11. For each decade, compute the company with maximum revenue

In [445]:
revenue_grouped_m = revenue[['company_id', 'decade','revenue/companies']].groupby(['decade', 'company_id']).sum().rename(index=str, columns={'revenue/companies':'overall_revenue'})
revenue_grouped_m = revenue_grouped_m.groupby('decade').max()
revenue_grouped_m = pd.merge(revenue_grouped_m.reset_index(), revenue_grouped.reset_index(), on=['decade', 'overall_revenue'])
revenue_grouped_m.head(20)

,decade,overall_revenue,company_id,company_name
0,1910.0,4.197376e+06,1307.0,Triangle Film Corporation
1,1910.0,4.197376e+06,1308.0,Wark Producing Corp.
2,1920.0,2.635800e+07,8411.0,Metro-Goldwyn-Mayer (MGM)
3,1930.0,2.000882e+08,1553.0,Selznick International Pictures
4,1930.0,2.000882e+08,8411.0,Metro-Goldwyn-Mayer (MGM)
5,1940.0,3.095972e+08,3166.0,Walt Disney Productions
6,1950.0,6.850000e+07,306.0,Twentieth Century Fox Film Corporation
7,1960.0,3.072341e+08,7576.0,Eon Productions
8,1970.0,7.673303e+08,33.0,Universal Pictures
9,1980.0,3.082731e+09,4.0,Paramount Pictures


### 12. In each year, how many movies have revenue smaller than the budget?

In [462]:
movies_negative = movie_id.loc[movie_id.gross_margin < 0, ['years','gross_margin']].groupby('years').count().rename(index=str, columns={'gross_margin': 'count_unsuccess'})
movies_negative.tail(10)


,count_unsuccess
years,
2007.0,55
2008.0,64
2009.0,78
2010.0,65
2011.0,70
2012.0,52
2013.0,62
2014.0,59
2015.0,67


,years,gross_margin
movie_id,,
57201,2013,-165710090
2698,2007,-2000000


In [50]:
movies.loc[movies['years'] == 'Missing']

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,years
4553,0,[],NaN,380097,[],en,America Is Still the Place,1971 post civil rights San Francisco seemed li...,0.0,[],...,NaN,0,0.0,[],Released,NaN,America Is Still the Place,0.0,0,Missing


In [ ]:
rev = movie_id.reset_index().groupby(['language_name',]).count()


$e^{i\pi} + 1 = 0$

$$e^x=\sum_{i=0}^\infty \frac{1}{i!}x^i$$

- One
- Sublist
- This